## Data reading and Cleaning

Data and more detailed explanations of the data can be found at the following websites:

https://www.basketball-reference.com/leagues/NBA_2023_advanced.html 


https://www.basketball-reference.com/contracts/players.html

In [1]:
# Read in the data

using CSV, DataFrames
data = DataFrame(CSV.File("C:\\Users\\tfurr\\OneDrive\\Documents\\optprojectdata.csv"))
size(data)

(459, 34)

In [2]:
# Only keep necessary columns. Also makes sure we drop duplicate players that show up

data = data[:,1:34]
data = unique(data[completecases(data), :])
data[1:5,:]

Row,PlayerID,Player,Pos,Age,Team,G,MP,MinM,PER,TS_perc,X3PAr,FTr,ORB_perc,DRB_perc,TRB_perc,AST_perc,STL_perc,BLK_perc,TOV_perc,USG_perc,OWS,DWS,WS,WS_per_48,OBPM,DBPM,BPM,VORP,X2022_23,C,SG,SF,PG,PF
,String15,String31,String7,Int64,String3,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64?,Int64,Int64,Int64,Int64,Int64
1,jokicni01,Nikola Jokic,C,27,DEN,51,1714,408,31.8,0.703,0.145,0.414,7.4,31.6,19.8,47.3,1.8,1.9,16.8,27.3,8.6,2.7,11.3,0.318,8.6,4.6,13.2,6.6,33047803,1,0,0,0,0
2,embiijo01,Joel Embiid,C,28,PHI,45,1571,360,30.6,0.646,0.153,0.564,6.0,27.9,17.1,22.3,1.6,4.0,12.1,37.2,5.3,2.8,8.1,0.246,6.3,2.1,8.4,4.1,33616770,1,0,0,0,0
3,doncilu01,Luka Doncic,PG,23,DAL,50,1826,400,30.2,0.614,0.353,0.499,2.9,26.1,14.3,44.5,2.1,1.2,11.9,38.3,6.3,2.4,8.6,0.226,8.2,1.9,10.1,5.5,37096500,0,0,0,1,0
4,antetgi01,Giannis Antetokounmpo,PF,28,MIL,47,1554,376,28.4,0.595,0.143,0.617,7.5,30.0,19.1,31.4,1.1,2.0,13.0,39.0,3.5,2.9,6.4,0.197,5.4,2.5,7.9,3.9,42492492,0,0,0,0,1
5,davisan02,Anthony Davis,C,29,LAL,35,1168,280,28.4,0.628,0.085,0.417,10.8,28.1,19.7,12.7,1.7,5.1,9.3,28.8,3.4,2.0,5.4,0.224,4.6,1.7,6.3,2.4,37980720,1,0,0,0,0


In [3]:
# Some players names are spelled wrong or Julia has some difficulty with non-English characters

data.Player = replace.(data.Player, "Kawhi Leord" => "Kawhi Leonard")
data.Player = replace.(data.Player, "Willy Herng\xf3mez" => "Willy Hernangomez")

441-element Vector{String}:
 "Nikola Jokic"
 "Joel Embiid"
 "Luka Doncic"
 "Giannis Antetokounmpo"
 "Anthony Davis"
 "Shai Gilgeous-Alexander"
 "Damian Lillard"
 "Kevin Durant"
 "Jaden Springer"
 "Boban Marjanovic"
 "Jimmy Butler"
 "Zion Williamson"
 "LeBron James"
 ⋮
 "Kessler Edwards"
 "Blake Wesley"
 "Johnny Davis"
 "Thasis Antetokounmpo"
 "Danny Green"
 "Ryan Arcidiacono"
 "Ryan Rollins"
 "Isaiah Todd"
 "Udonis Haslem"
 "Chima Moneke"
 "Leandro Bolmaro"
 "Marko Simonovic"

In [4]:
# The main data frame we will deal with keeps these variables. MinM is minimum number of minuts that need to be played to qualify

df = data[:, [:PlayerID, :Player, :Pos, :MP, :MinM, :PER, :X2022_23, :C, :SG, :SF, :PG, :PF]]
df[1:5,:]

Row,PlayerID,Player,Pos,MP,MinM,PER,X2022_23,C,SG,SF,PG,PF
,String15,String,String7,Int64,Int64,Float64,Int64?,Int64,Int64,Int64,Int64,Int64
1,jokicni01,Nikola Jokic,C,1714,408,31.8,33047803,1,0,0,0,0
2,embiijo01,Joel Embiid,C,1571,360,30.6,33616770,1,0,0,0,0
3,doncilu01,Luka Doncic,PG,1826,400,30.2,37096500,0,0,0,1,0
4,antetgi01,Giannis Antetokounmpo,PF,1554,376,28.4,42492492,0,0,0,0,1
5,davisan02,Anthony Davis,C,1168,280,28.4,37980720,1,0,0,0,0


In [5]:
# Functions that will help us create our tables of players that are included in our model

function find_players_binary(vector)
    playerlist = []
    for i in 1:length(vector)
        if vector[i] == 1
            push!(playerlist, i)
        end
    end
    return playerlist
end

function find_players_string(include_players)
    playerlist = []
    for player in include_players
        player_index = findfirst(x -> occursin(player, x), df.Player)
        push!(playerlist, player_index)
    end
    return playerlist
end

find_players_string (generic function with 1 method)

In [6]:
# see if a player is located in the df
my_player_index = findfirst(x -> occursin("Russell Westbrook", x), df.Player)

# print the index
println("This player is located at index $my_player_index.")

This player is located at index 139.


In [7]:
search_player = filter(row -> contains(row.Player, "Russell Westbrook"), df)

Row,PlayerID,Player,Pos,MP,MinM,PER,X2022_23,C,SG,SF,PG,PF
,String15,String,String7,Int64,Int64,Float64,Int64?,Int64,Int64,Int64,Int64,Int64
1,westbru01,Russell Westbrook,PG,1491,416,15.5,47063478,0,0,0,1,0


## Main model

In [8]:
# include whatever players you want to this list
include_players = []

# list of positions
positions = ["C", "SG", "SF", "PG", "PF"]

# establish budget
budget = 134000000

134000000

In [9]:
using JuMP, GLPK;

model = Model(GLPK.Optimizer)

# If you only want to include a subset of players, change n
n = nrow(df)

# c is the column vector of PER
c = df[1:n, 6]

# a is the column vector of salaries
a = df[1:n, 7]

# matrix of positions
mat = df[1:n ,8:12]

# Minimum minutes that need to be played to qualify to be in the model
min_m = df[!,5]

# Minutes played for each player
mp = df[!,4]

@variable(model, x[i=1:n], Bin)

# add requirements for players to include
for player in include_players
    player_index = findfirst(x -> occursin(player, x), df.Player)
    @constraint(model, sum(x[player_index]) == 1)
end

# limiting to no more than 5 per position
for p in positions
    pos = df[!, p]
    @constraint(model, sum(pos[i]*x[i] for i = 1:n) <= 5)
end

# ensuring that players included have played enough games/minutes
for i = 1:n
    @constraint(model, mp[i] >= min_m[i]*x[i])
end


# budget
@constraint(model, sum(a[i]*x[i] for i = 1:n) <= budget)

# 14 <= number of players <= 15
@constraint(model, sum(x[i] for i=1:n) <= 15)
@constraint(model, sum(x[i] for i=1:n) >= 14)

# Makes sure we have at least 2 players from each position
for j = 1:5
    @constraint(model, sum(x[i]*mat[i, j] for i=1:n) >= 2)
end

@objective(model, Max, sum(c[j]*x[j] for j=1:n))

31.8 x[1] + 30.6 x[2] + 30.2 x[3] + 28.4 x[4] + 28.4 x[5] + 27.3 x[6] + 26.6 x[7] + 26.5 x[8] + 26.1 x[9] + 25.9 x[10] + 25.4 x[11] + 25.3 x[12] + 24.9 x[13] + 24.8 x[14] + 24 x[15] + 23.7 x[16] + 23.7 x[17] + 23 x[18] + 23 x[19] + 22.9 x[20] + 22.7 x[21] + 22.7 x[22] + 22.5 x[23] + 22.5 x[24] + 22.3 x[25] + 22 x[26] + 21.9 x[27] + 21.8 x[28] + 21.6 x[29] + 21.5 x[30] + 21.5 x[31] + 21.4 x[32] + 21.3 x[33] + 21.2 x[34] + 21.2 x[35] + 21.2 x[36] + 21.2 x[37] + 21.1 x[38] + 21 x[39] + 21 x[40] + 21 x[41] + 20.9 x[42] + 20.9 x[43] + 20.8 x[44] + 20.8 x[45] + 20.7 x[46] + 20.6 x[47] + 20.5 x[48] + 20.5 x[49] + 20.4 x[50] + 20.3 x[51] + 20.2 x[52] + 20 x[53] + 19.9 x[54] + 19.7 x[55] + 19.7 x[56] + 19.5 x[57] + 19.3 x[58] + 19.3 x[59] + 19.2 x[60] + 19.2 x[61] + 19.2 x[62] + 19 x[63] + 18.9 x[64] + 18.9 x[65] + 18.7 x[66] + 18.7 x[67] + 18.6 x[68] + 18.6 x[69] + 18.4 x[70] + 18.1 x[71] + 18 x[72] + 18 x[73] + 17.8 x[74] + 17.8 x[75] + 17.8 x[76] + 17.7 x[77] + 17.6 x[78] + 17.6 x[79] + 17.6

In [34]:
JuMP.optimize!(model)

println("Objective value: ", JuMP.objective_value(model))
println("Average PER: ", (JuMP.objective_value(model))/15)

Objective value: 327.9
Average PER: 21.86


In [11]:
# This gives us our team

vector = JuMP.value.(x)
guess = df[find_players_binary(vector), [2,3,6,7]]

Row,Player,Pos,PER,X2022_23
,String,String7,Float64,Int64?
1,Nikola Jokic,C,31.8,33047803
2,Shai Gilgeous-Alexander,PG,27.3,30913750
3,Zion Williamson,PF,25.3,13534817
4,Ja Morant,PG,23.7,12119440
5,Tyrese Haliburton,PG,22.7,4215120
6,Lauri Markkanen,SF,22.5,16475454
7,Thomas Bryant,C,21.5,1836090
8,Andre Drummond,C,21.5,3200000
9,Willy Hernangomez,C,21.2,2443581


In [12]:
total_salary = sum(guess[:, 4]) # The total salary is under the salary cap

132750326

# Clippers Example

In [13]:
include_players = ["Nicolas Batum", 
                   "Brandon Boston Jr.", 
                   "Amir Coffey", 
                   "Robert Covington", 
                   "Paul George",
                   "Eric Gordon",
                   "Bones Hyland",
                   "Kawhi Leonard",
                   "Terance Mann",
                   "Marcus Morris",
                   "Mason Plumlee",
                   "Norman Powell",
                   "Jason Preston",
                   "Russell Westbrook"]

14-element Vector{String}:
 "Nicolas Batum"
 "Brandon Boston Jr."
 "Amir Coffey"
 "Robert Covington"
 "Paul George"
 "Eric Gordon"
 "Bones Hyland"
 "Kawhi Leonard"
 "Terance Mann"
 "Marcus Morris"
 "Mason Plumlee"
 "Norman Powell"
 "Jason Preston"
 "Russell Westbrook"

In [14]:
clippers_roster = df[find_players_string(include_players), [2,3,6,7]]

Row,Player,Pos,PER,X2022_23
,String,String7,Float64,Int64?
1,Nicolas Batum,PF,10.8,19700319
2,Brandon Boston Jr.,SG,13.7,1563518
3,Amir Coffey,SG,7.6,3395062
4,Robert Covington,PF,13.1,12307692
5,Paul George,SG,19.7,42492492
6,Eric Gordon,SG,11.6,19568360
7,Bones Hyland,PG,14.6,2201520
8,Kawhi Leonard,SF,23.7,42492492
9,Terance Mann,SG,14.1,1930681


In [15]:
sum(clippers_roster[:, 4]) # Check the current salary amount

236490263

In [16]:
budget = 250000000

250000000

In [17]:
clippers = Model(GLPK.Optimizer)

# Below variables are the same as above 
n = nrow(df)
c = df[1:n, 6]
a = df[1:n, 7]
mat = df[1:n ,8:12]

min_m = df[!,5]
mp = df[!,4]

@variable(clippers, x[i=1:n], Bin)

# add requirements for players to include
for player in include_players
    player_index = findfirst(x -> occursin(player, x), df.Player)
    @constraint(clippers, sum(x[player_index]) == 1)
end

# budget
@constraint(clippers, sum(a[i]*x[i] for i = 1:n) <= budget)

# 14 <= number of players <= 15
@constraint(clippers, sum(x[i] for i=1:n) <= 15)
@constraint(clippers, sum(x[i] for i=1:n) >= 14)


# adjusting number of players per position
for p in positions
    pos = df[!, p]
    @constraint(clippers, sum(pos[i]*x[i] for i = 1:n) <= 6)
    @constraint(clippers, sum(pos[i]*x[i] for i = 1:n) >= 0)
end

for i = 1:n
    @constraint(clippers, mp[i] >= min_m[i]*x[i])
end

@objective(clippers, Max, sum(c[j]*x[j] for j=1:n))

31.8 x[1] + 30.6 x[2] + 30.2 x[3] + 28.4 x[4] + 28.4 x[5] + 27.3 x[6] + 26.6 x[7] + 26.5 x[8] + 26.1 x[9] + 25.9 x[10] + 25.4 x[11] + 25.3 x[12] + 24.9 x[13] + 24.8 x[14] + 24 x[15] + 23.7 x[16] + 23.7 x[17] + 23 x[18] + 23 x[19] + 22.9 x[20] + 22.7 x[21] + 22.7 x[22] + 22.5 x[23] + 22.5 x[24] + 22.3 x[25] + 22 x[26] + 21.9 x[27] + 21.8 x[28] + 21.6 x[29] + 21.5 x[30] + 21.5 x[31] + 21.4 x[32] + 21.3 x[33] + 21.2 x[34] + 21.2 x[35] + 21.2 x[36] + 21.2 x[37] + 21.1 x[38] + 21 x[39] + 21 x[40] + 21 x[41] + 20.9 x[42] + 20.9 x[43] + 20.8 x[44] + 20.8 x[45] + 20.7 x[46] + 20.6 x[47] + 20.5 x[48] + 20.5 x[49] + 20.4 x[50] + 20.3 x[51] + 20.2 x[52] + 20 x[53] + 19.9 x[54] + 19.7 x[55] + 19.7 x[56] + 19.5 x[57] + 19.3 x[58] + 19.3 x[59] + 19.2 x[60] + 19.2 x[61] + 19.2 x[62] + 19 x[63] + 18.9 x[64] + 18.9 x[65] + 18.7 x[66] + 18.7 x[67] + 18.6 x[68] + 18.6 x[69] + 18.4 x[70] + 18.1 x[71] + 18 x[72] + 18 x[73] + 17.8 x[74] + 17.8 x[75] + 17.8 x[76] + 17.7 x[77] + 17.6 x[78] + 17.6 x[79] + 17.6

In [18]:
JuMP.optimize!(clippers)

println("Objective value: ", JuMP.objective_value(clippers))

println("Average PER:", (JuMP.objective_value(clippers))/15)

Objective value: 228.99999999999997
Average PER:15.266666666666664


In [19]:
# Here is our team. Note that Ja Morant is the player that was added

vector = JuMP.value.(x)
guess = df[find_players_binary(vector), [2,3,6,7]]

Row,Player,Pos,PER,X2022_23
,String,String7,Float64,Int64?
1,Kawhi Leonard,SF,23.7,42492492
2,Ja Morant,PG,23.7,12119440
3,Mason Plumlee,C,20.3,9080417
4,Paul George,SG,19.7,42492492
5,Norman Powell,SG,18.1,16758621
6,Russell Westbrook,PG,15.5,47063478
7,Bones Hyland,PG,14.6,2201520
8,Terance Mann,SG,14.1,1930681
9,Brandon Boston Jr.,SG,13.7,1563518


In [20]:
sum(guess[:,4]) #Budget works

248609703

# Suns Example

In [21]:
include_players = ["Devin Booker", 
                   "Kevin Durant", 
                   "Chris Paul"]

3-element Vector{String}:
 "Devin Booker"
 "Kevin Durant"
 "Chris Paul"

In [22]:
suns_roster = df[find_players_string(include_players), [2,3,6,7]]

Row,Player,Pos,PER,X2022_23
,String,String7,Float64,Int64?
1,Devin Booker,SG,21.3,33833400
2,Kevin Durant,PF,26.5,44119845
3,Chris Paul,PG,17.6,28400000


In [23]:
budget = 150000000

150000000

In [24]:
suns = Model(GLPK.Optimizer)
n = nrow(df)
c = df[1:n, 6]
a = df[1:n, 7]
mat = df[1:n ,8:12]

min_m = df[!,5]
mp = df[!,4]

@variable(suns, x[i=1:n], Bin)

# add requirements for players to include
for player in include_players
    player_index = findfirst(x -> occursin(player, x), df.Player)
    @constraint(suns, sum(x[player_index]) == 1)
end

# budget
@constraint(suns, sum(a[i]*x[i] for i = 1:n) <= budget)

# 14 <= number of players <= 15
@constraint(suns, sum(x[i] for i=1:n) <= 15)
@constraint(suns, sum(x[i] for i=1:n) >= 14)


# limiting to no more than 5 per position
for p in positions
    pos = df[!, p]
    @constraint(suns, sum(pos[i]*x[i] for i = 1:n) <= 5)
    @constraint(suns, sum(pos[i]*x[i] for i = 1:n) >= 2)
end

for i = 1:n
    @constraint(suns, mp[i] >= min_m[i]*x[i])
end

@objective(suns, Max, sum(c[j]*x[j] for j=1:n))

31.8 x[1] + 30.6 x[2] + 30.2 x[3] + 28.4 x[4] + 28.4 x[5] + 27.3 x[6] + 26.6 x[7] + 26.5 x[8] + 26.1 x[9] + 25.9 x[10] + 25.4 x[11] + 25.3 x[12] + 24.9 x[13] + 24.8 x[14] + 24 x[15] + 23.7 x[16] + 23.7 x[17] + 23 x[18] + 23 x[19] + 22.9 x[20] + 22.7 x[21] + 22.7 x[22] + 22.5 x[23] + 22.5 x[24] + 22.3 x[25] + 22 x[26] + 21.9 x[27] + 21.8 x[28] + 21.6 x[29] + 21.5 x[30] + 21.5 x[31] + 21.4 x[32] + 21.3 x[33] + 21.2 x[34] + 21.2 x[35] + 21.2 x[36] + 21.2 x[37] + 21.1 x[38] + 21 x[39] + 21 x[40] + 21 x[41] + 20.9 x[42] + 20.9 x[43] + 20.8 x[44] + 20.8 x[45] + 20.7 x[46] + 20.6 x[47] + 20.5 x[48] + 20.5 x[49] + 20.4 x[50] + 20.3 x[51] + 20.2 x[52] + 20 x[53] + 19.9 x[54] + 19.7 x[55] + 19.7 x[56] + 19.5 x[57] + 19.3 x[58] + 19.3 x[59] + 19.2 x[60] + 19.2 x[61] + 19.2 x[62] + 19 x[63] + 18.9 x[64] + 18.9 x[65] + 18.7 x[66] + 18.7 x[67] + 18.6 x[68] + 18.6 x[69] + 18.4 x[70] + 18.1 x[71] + 18 x[72] + 18 x[73] + 17.8 x[74] + 17.8 x[75] + 17.8 x[76] + 17.7 x[77] + 17.6 x[78] + 17.6 x[79] + 17.6

In [25]:
JuMP.optimize!(suns)

println("Objective value: ", JuMP.objective_value(suns))

println("Average PER:", (JuMP.objective_value(suns))/15)

Objective value: 306.5
Average PER:20.433333333333334


In [26]:
# Here is our team

vector = JuMP.value.(x)
guess = df[find_players_binary(vector), [2,3,6,7]]

Row,Player,Pos,PER,X2022_23
,String,String7,Float64,Int64?
1,Kevin Durant,PF,26.5,44119845
2,Zion Williamson,PF,25.3,13534817
3,Tyrese Haliburton,PG,22.7,4215120
4,Thomas Bryant,C,21.5,1836090
5,Andre Drummond,C,21.5,3200000
6,Devin Booker,SG,21.3,33833400
7,Willy Hernangomez,C,21.2,2443581
8,Walker Kessler,C,21.0,2696400
9,Daniel Gafford,C,20.5,1930681


In [27]:
sum(guess[:,4]) # Salary is good

148424336

# Warriors Example

In [28]:
include_players = ["Stephen Curry"]

not_include_players = ["Klay Thompson"]

1-element Vector{String}:
 "Klay Thompson"

In [29]:
budget = 150000000

150000000

In [30]:
warriors = Model(GLPK.Optimizer)
n = nrow(df)
c = df[1:n, 6]
a = df[1:n, 7]
mat = df[1:n ,8:12]
CENTER = df[1:n,8]
POWER_F = df[1:n,12]

min_m = df[!,5]
mp = df[!,4]

@variable(warriors, x[i=1:n], Bin)

# add requirements for players to include
for player in include_players
    player_index = findfirst(x -> occursin(player, x), df.Player)
    @constraint(warriors, sum(x[player_index]) == 1)
end

# add requirements for players to NOT include
for player in not_include_players
    player_index = findfirst(x -> occursin(player, x), df.Player)
    @constraint(warriors, sum(x[player_index]) == 0)
end

# budget
@constraint(warriors, sum(a[i]*x[i] for i = 1:n) <= budget)

# 14 <= number of players <= 15
@constraint(warriors, sum(x[i] for i=1:n) <= 15)
@constraint(warriors, sum(x[i] for i=1:n) >= 14)


# sum of CENTER column is 0 to make sure we include 0 centers
@constraint(warriors, sum(CENTER[i]*x[i] for i=1:n) == 0)

# sum of POWER_F column is 0 to make sure we include 0 power forwards
@constraint(warriors, sum(POWER_F[i]*x[i] for i=1:n) == 0)

for i = 1:n
    @constraint(warriors, mp[i] >= min_m[i]*x[i])
end

@objective(warriors, Max, sum(c[j]*x[j] for j=1:n))

31.8 x[1] + 30.6 x[2] + 30.2 x[3] + 28.4 x[4] + 28.4 x[5] + 27.3 x[6] + 26.6 x[7] + 26.5 x[8] + 26.1 x[9] + 25.9 x[10] + 25.4 x[11] + 25.3 x[12] + 24.9 x[13] + 24.8 x[14] + 24 x[15] + 23.7 x[16] + 23.7 x[17] + 23 x[18] + 23 x[19] + 22.9 x[20] + 22.7 x[21] + 22.7 x[22] + 22.5 x[23] + 22.5 x[24] + 22.3 x[25] + 22 x[26] + 21.9 x[27] + 21.8 x[28] + 21.6 x[29] + 21.5 x[30] + 21.5 x[31] + 21.4 x[32] + 21.3 x[33] + 21.2 x[34] + 21.2 x[35] + 21.2 x[36] + 21.2 x[37] + 21.1 x[38] + 21 x[39] + 21 x[40] + 21 x[41] + 20.9 x[42] + 20.9 x[43] + 20.8 x[44] + 20.8 x[45] + 20.7 x[46] + 20.6 x[47] + 20.5 x[48] + 20.5 x[49] + 20.4 x[50] + 20.3 x[51] + 20.2 x[52] + 20 x[53] + 19.9 x[54] + 19.7 x[55] + 19.7 x[56] + 19.5 x[57] + 19.3 x[58] + 19.3 x[59] + 19.2 x[60] + 19.2 x[61] + 19.2 x[62] + 19 x[63] + 18.9 x[64] + 18.9 x[65] + 18.7 x[66] + 18.7 x[67] + 18.6 x[68] + 18.6 x[69] + 18.4 x[70] + 18.1 x[71] + 18 x[72] + 18 x[73] + 17.8 x[74] + 17.8 x[75] + 17.8 x[76] + 17.7 x[77] + 17.6 x[78] + 17.6 x[79] + 17.6

In [31]:
JuMP.optimize!(warriors)

println("Objective value: ", JuMP.objective_value(warriors))

println("Average PER:", (JuMP.objective_value(warriors))/15)

Objective value: 290.7
Average PER:19.38


In [32]:
# This is our team. Notice it is all PG, SG and SF. Mainly 3-point shooters

vector = JuMP.value.(x)
guess = df[find_players_binary(vector), [2,3,6,7]]

Row,Player,Pos,PER,X2022_23
,String,String7,Float64,Int64?
1,Luka Doncic,PG,30.2,37096500
2,Stephen Curry,PG,24.8,48070014
3,Ja Morant,PG,23.7,12119440
4,Tyrese Haliburton,PG,22.7,4215120
5,Lauri Markkanen,SF,22.5,16475454
6,Darius Garland,PG,19.7,8920795
7,Desmond Bane,SG,19.2,2130240
8,Isaiah Joe,SG,17.0,1836090
9,Devin Vassell,SG,16.6,4437000


In [33]:
sum(guess[:,4]) # Total salary for this team. It stays under the amount we said

149552671